In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
tf.__version__

'1.15.0'

In [3]:
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)
sess = tf.Session()
output = sess.run(add)
print(output) # 12
sess.close()

12


In [4]:
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)
with tf.Session() as sess:
    output = sess.run(add)
    print(output) # 12

12


In [5]:
import numpy as np
a_n = np.array(5)
b_n = np.array(7)
output_n = np.add(a_n, b_n)
print(output_n) # 12

12


In [6]:
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)

In [7]:
print(a) # Tensor("Const:0", shape=(), dtype=int32)
print(add) # Tensor("Add:0", shape=(), dtype=int32)

Tensor("Const_4:0", shape=(), dtype=int32)
Tensor("Add_2:0", shape=(), dtype=int32)


In [8]:
sess = tf.Session()

In [9]:
output = sess.run(add)
print(output) # 12

12


In [10]:
c = tf.placeholder(tf.int32)
d = tf.placeholder(tf.int32)
add = tf.add(c, d)
sess = tf.Session()
with tf.Session() as sess:
    output = sess.run(add, feed_dict={c:5, d:7})
    print(output) # 12

12


In [11]:
with tf.Session() as sess:
    output = sess.run(add, feed_dict={c:20, d:32})
    print(output) # 52

52


In [12]:
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

In [13]:
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

In [14]:
W = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(tf.zeros([1]))

In [15]:
y = tf.sigmoid(tf.matmul(x, W) + b)
cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1 - t) * tf.log(1 - y))

In [16]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [17]:
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [19]:
for epoch in range(1000):
    sess.run(train_step, feed_dict={x:x_train, t:y_train})
# 100回ごとに正解率を表示
    if epoch % 100 == 0:
        acc_val = sess.run(accuracy, feed_dict={x:x_train, t:y_train})
        print ('epoch: %d, Accuracy: %f'%(epoch, acc_val))

epoch: 0, Accuracy: 0.750000
epoch: 100, Accuracy: 1.000000
epoch: 200, Accuracy: 1.000000
epoch: 300, Accuracy: 1.000000
epoch: 400, Accuracy: 1.000000
epoch: 500, Accuracy: 1.000000
epoch: 600, Accuracy: 1.000000
epoch: 700, Accuracy: 1.000000
epoch: 800, Accuracy: 1.000000
epoch: 900, Accuracy: 1.000000


In [20]:
#学習結果が正しいか確認
classified = sess.run(correct_prediction, feed_dict={x:x_train, t:y_train})
#出力yの確認
prob = sess.run(y, feed_dict={x:x_train, t:y_train})
print(classified)
# [[ True]
# [ True]
# [ True]
# [ True]]
print(prob)
# [[  1.96514215e-04]
# [  4.90498319e-02]
# [  4.90498319e-02]
# [  9.31203783e-01]]

[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9651403e-04]
 [4.9049813e-02]
 [4.9049813e-02]
 [9.3120378e-01]]


In [21]:
print('W:', sess.run(W))
print('b:', sess.run(b))
# W: [[ 5.5699544]
# [ 5.5699544]]
# b: [-8.53457928]

W: [[5.569955]
 [5.569955]]
b: [-8.53458]


In [22]:
mat = tf.matmul(x, W)
y = tf.sigmoid(mat + b)
print(sess.run(mat, feed_dict={x:x_train, t:y_train}))

[[ 0.      ]
 [ 5.569955]
 [ 5.569955]
 [11.13991 ]]


In [23]:
sess.close()

# 【問題1】スクラッチを振り返る

- 重みを初期化する必要があった
- エポックのループが必要だった
- optimizerで重みやバイアスを更新する必要があった
- コスト関数を計算する必要があった

# 【問題2】スクラッチとTensorFlowの対応を考える

In [24]:
from sklearn.datasets import load_iris

In [25]:
data = load_iris()

In [26]:
y = data.target[50:] - 1 
X = data.data[50:]
y = y.astype(np.int)[:, np.newaxis]

In [27]:
tf.reset_default_graph()
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 0, loss : 0.0067, val_loss : 0.3707, acc : 1.000, val_acc : 0.812
Epoch 1, loss : 0.0003, val_loss : 0.6963, acc : 1.000, val_acc : 0.875
Epoch 2, loss : 0.0909, val_loss : 3.2909, acc : 1.000, val_acc : 0.688
Epoch 3, loss : 0.0019, val_loss : 2.7392, acc : 1.000, val_acc : 0.750
Epoch 4, loss : 0.0000, val_loss : 0.3515, acc : 1.000, val_acc : 0.938
Epoch 5, loss : 0.0000, val_loss : 0.5249, acc : 1.000, val_acc : 0.938
Epoch 6, loss : 1.1963, val_loss : 6.6446, acc : 0.750, val_acc : 0.688
Epoch 7, loss : 0.0000, val_loss : 0.6408, acc : 1.000, val_acc : 0.938
Epoch 8, loss : 0.0000, val_loss : 1.0787, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 2.6460, val_loss : 9.0949, acc : 0.500, val_acc : 0.625
test_acc : 0.550


## 重みとバイアスの初期化を下記コードで行っている
```python
# variableの初期化
init = tf.global_variables_initializer()
```
## エポックのループを下記コードで行っている
```python
for epoch in range(num_epochs):
    # エポックごとにループ
    total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
    total_loss = 0
    total_acc = 0
```
## 重みやバイアスの更新を下記コードで行っている
```python
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
```
## コスト関数の計算を下記コードで行っている
```python
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
```

# 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成

In [28]:
y = data.target
X = data.data
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [29]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [31]:
tf.reset_default_graph()
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 20
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("int32", [None, ])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.nn.in_top_k(logits, Y, 1)
y_pred = tf.argmax(logits, 1)
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()
saver = tf.train.Saver()
loss_summary = tf.summary.scalar('cross_entropy', loss_op)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
            summary_str = loss_summary.eval(feed_dict={X: mini_batch_x, Y: mini_batch_y})
            file_writer.add_summary(summary_str, i)
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))
    save_path = saver.save(sess, 'model_iris.ckpt')

Epoch 0, loss : 13.1371, val_loss : 9.4849, acc : 0.688, val_acc : 0.708
Epoch 1, loss : 4.9850, val_loss : 4.9620, acc : 0.750, val_acc : 0.792
Epoch 2, loss : 7.0846, val_loss : 5.2907, acc : 0.688, val_acc : 0.708
Epoch 3, loss : 2.3826, val_loss : 2.0843, acc : 0.875, val_acc : 0.833
Epoch 4, loss : 2.5591, val_loss : 2.1034, acc : 0.812, val_acc : 0.750
Epoch 5, loss : 0.2750, val_loss : 0.1097, acc : 0.938, val_acc : 0.958
Epoch 6, loss : 0.2612, val_loss : 1.2066, acc : 0.938, val_acc : 0.917
Epoch 7, loss : 0.0023, val_loss : 0.0576, acc : 1.000, val_acc : 0.958
Epoch 8, loss : 0.0001, val_loss : 0.7045, acc : 1.000, val_acc : 0.917
Epoch 9, loss : 0.0000, val_loss : 0.2689, acc : 1.000, val_acc : 0.958
test_acc : 0.933


In [32]:
with tf.Session() as sess:
    saver.restore(sess, save_path)
    print(sess.run(y_pred, feed_dict={X: X_test, Y:y_test}))
print(y_test)

INFO:tensorflow:Restoring parameters from model_iris.ckpt
[2 1 0 2 0 2 0 1 1 1 1 1 1 1 1 0 1 1 0 0 2 1 1 0 2 0 0 1 1 0]
[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0]


In [33]:
file_writer.close()

# 【問題4】House Pricesのモデルを作成

In [34]:
df = pd.read_csv('train.csv')

In [35]:
y = np.array(df['SalePrice'])
X = np.array(df[['GrLivArea', 'YearBuilt']])
y = np.log(y)
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
scaler= StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [36]:
tf.reset_default_graph()
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 20
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, ])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.square(logits - Y))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
y_pred = logits
# variableの初期化
init = tf.global_variables_initializer()
saver = tf.train.Saver()
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, loss, val_loss))
    test_loss = sess.run(loss_op, feed_dict={X: X_test, Y: y_test})
    print("test_loss : {:.3f}".format(test_loss))
    save_path = saver.save(sess, 'model_house.ckpt')

Epoch 0, loss : 11.0523, val_loss : 16.8028
Epoch 1, loss : 1.6119, val_loss : 2.8990
Epoch 2, loss : 1.1176, val_loss : 1.5287
Epoch 3, loss : 0.8882, val_loss : 1.1103
Epoch 4, loss : 0.8755, val_loss : 0.8683
Epoch 5, loss : 0.7860, val_loss : 0.7276
Epoch 6, loss : 0.6848, val_loss : 0.6267
Epoch 7, loss : 0.6246, val_loss : 0.5767
Epoch 8, loss : 0.5697, val_loss : 0.5466
Epoch 9, loss : 0.5056, val_loss : 0.5394
test_loss : 0.657


In [37]:
with tf.Session() as sess:
    saver.restore(sess, save_path)
    print(sess.run(y_pred, feed_dict={X: X_test[:10], Y: y_test[:10]}))
print(y_test[:10])

INFO:tensorflow:Restoring parameters from model_house.ckpt
[[11.570521]
 [12.579294]
 [11.66625 ]
 [10.515232]
 [11.932314]
 [11.15009 ]
 [11.859799]
 [12.058229]
 [ 9.228918]
 [11.969879]]
[12.20918779 11.79810441 11.60823564 12.16525065 11.38509209 11.35040654
 12.55292652 11.85651517 13.5211395  11.9103584 ]


# 【問題5】MNISTのモデルを作成

In [38]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [39]:
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

In [40]:
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0

1.0
0.0


In [41]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) # float64

(60000,)
(60000, 10)
float64


In [42]:
# trainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [43]:
tf.reset_default_graph()
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 20
num_epochs = 10
n_hidden1 = 400
n_hidden2 = 200
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("int32", [None, ])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.nn.in_top_k(logits, Y, 1)
y_pred = tf.argmax(logits, 1)
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()
saver = tf.train.Saver()
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))
    save_path = saver.save(sess, 'model_mnist.ckpt')

Epoch 0, loss : 2.0637, val_loss : 1.2092, acc : 0.550, val_acc : 0.649
Epoch 1, loss : 0.9944, val_loss : 0.8813, acc : 0.700, val_acc : 0.814
Epoch 2, loss : 0.4852, val_loss : 0.6481, acc : 0.850, val_acc : 0.850
Epoch 3, loss : 0.5420, val_loss : 0.6750, acc : 0.850, val_acc : 0.846
Epoch 4, loss : 0.4145, val_loss : 0.5841, acc : 0.850, val_acc : 0.862
Epoch 5, loss : 0.5491, val_loss : 0.9263, acc : 0.800, val_acc : 0.845
Epoch 6, loss : 0.5044, val_loss : 0.5429, acc : 0.850, val_acc : 0.880
Epoch 7, loss : 0.4488, val_loss : 0.6241, acc : 0.850, val_acc : 0.906
Epoch 8, loss : 0.5094, val_loss : 0.9061, acc : 0.850, val_acc : 0.852
Epoch 9, loss : 0.7543, val_loss : 0.6769, acc : 0.800, val_acc : 0.822
test_acc : 0.820


In [44]:
with tf.Session() as sess:
    saver.restore(sess, save_path)
    print(sess.run(y_pred, feed_dict={X: X_test[:10], Y: y_test[:10]}))
print(y_test[:10])

INFO:tensorflow:Restoring parameters from model_mnist.ckpt
[7 9 1 0 4 1 4 9 9 9]
[7 2 1 0 4 1 4 9 5 9]
